In [37]:
import numpy as np
import pandas as pd

In [38]:
df = pd.read_csv('Data_set.csv')

In [39]:
df.head(3)

,Unnamed: 0,Data,Countries,Local,Industry Sector,Accident Level,Potential Accident Level,Genre,Employee or Third Party,Critical Risk,Description
0,0,2016-01-01 00:00:00,Country_01,Local_01,Mining,I,IV,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...
1,1,2016-01-02 00:00:00,Country_02,Local_02,Mining,I,IV,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...
2,2,2016-01-06 00:00:00,Country_01,Local_03,Mining,I,III,Male,Third Party (Remote),Manual Tools,In the sub-station MILPO located at level +170...


In [40]:
df['Critical Risk'].unique()

array(['Pressed', 'Pressurized Systems', 'Manual Tools', 'Others',
       'Fall prevention (same level)', 'Chemical substances',
       'Liquid Metal', 'Electrical installation', 'Confined space',
       'Pressurized Systems / Chemical Substances',
       'Blocking and isolation of energies', 'Suspended Loads', 'Poll',
       'Cut', 'Fall', 'Bees', 'Fall prevention', '\nNot applicable',
       'Traffic', 'Projection', 'Venomous Animals', 'Plates',
       'Projection/Burning', 'remains of choco',
       'Vehicles and Mobile Equipment', 'Projection/Choco',
       'Machine Protection', 'Power lock', 'Burn',
       'Projection/Manual Tools', 'Individual protection equipment',
       'Electrical Shock', 'Projection of fragments'], dtype=object)

<span style="color:green;font-weight:bold; font-size:1.5em">Extracting Dates from the text<span>

In [41]:
year =[]
month =[]
date=[]
for x in range(df.shape[0]):
    h = df['Data'][x].split()
    k = h[0].split('-')
    year.append(int(k[0]))
    month.append(int(k[1]))
    date.append(int(k[2]))


<span style="color:green;font-weight:bold; font-size:2em">Creating a DataFrame of Dates which will be concated with main DataFrame<span>

In [42]:
dates = list(zip(year,month,date))
df_date = pd.DataFrame(dates, columns = ['Year','Month','Date'])
df_date

,Year,Month,Date
0,2016,1,1
1,2016,1,2
2,2016,1,6
3,2016,1,8
4,2016,1,10
...,...,...,...
420,2017,7,4
421,2017,7,4
422,2017,7,5
423,2017,7,6


In [43]:
df_new =pd.concat([df,df_date],axis=1)


<span style="color:green;font-weight:bold; font-size:2em">Concating the Date with main DataFrame and dropping the unwanted columns<span>

In [44]:
df_new.drop(['Unnamed: 0','Data','Accident Level'],axis=1,inplace=True)

In [45]:
Potential_Accident_Level = {
    'I'   :   1,
    'II'  :   2,
    'III' :   3,
    'IV'  :   4,
    'V'   :   5,
    'VI'  :   6
}

In [46]:
Month_Modified = {
    
    1   :  'January',
    2   :  'February',
    3   :  'March',
    4   :  'April',
    5   :  'May',
    6   :  'June',
    7   :  'July',
    8   :  'August',
    9   :  'September',
    10  :  'October',
    11  :  'November',
    12  :  'December'
    
}

<span style="color:green;font-weight:bold; font-size:2em">Converting the Month Names and Potential Accident levels and dropping the original columns<span>

In [47]:
df_new['Potential_Accident_Level'] = df_new['Potential Accident Level'].map(Potential_Accident_Level)
df_new['Month_Modified'] = df_new['Month'].map(Month_Modified)

In [48]:
df_new.drop(['Potential Accident Level','Month'],axis=1,inplace=True)

<span style="color:green;font-weight:bold; font-size:2em">Converting the Day and Year into string because we will cocatenate them with the description<span>

In [84]:
df_new['Year'] = df_new['Year'].astype(str)
df_new['Date'] = df_new['Date'].astype(str)

<span style="color:green;font-weight:bold; font-size:2em">Extracting the Y Column and dropping it from our main DataFrame<span>

In [50]:
y=df_new['Potential_Accident_Level']
df_new.drop('Potential_Accident_Level', axis=1, inplace=True)

<span style="color:green;font-weight:bold; font-size:1em">In the below cell, we are just adding an extra SPACE at the end of each value of a cell in a row. This will be useful when at the end we will club all these values of cells of a row together. There is no immediate use of this. But when we will sum all the values of dataframes, this will play an important role as it will put space after each word<span>

In [51]:
for x in df_new.columns:
    df_new[x] = df_new[x] + ' '

<span style="color:green;font-weight:bold; font-size:1em">First, we will separate only the description part of our main Dataframe, we will clean it. Then we will add all other columns to this column to form a single column description or our X value<span>

In [52]:
df_new['Description']

0      While removing the drill rod of the Jumbo 08 f...
1      During the activation of a sodium sulphide pum...
2      In the sub-station MILPO located at level +170...
3      Being 9:45 am. approximately in the Nv. 1880 C...
4      Approximately at 11:45 a.m. in circumstances t...
                             ...                        
420    Being approximately 5:00 a.m. approximately, w...
421    The collaborator moved from the infrastructure...
422    During the environmental monitoring activity i...
423    The Employee performed the activity of strippi...
424    At 10:00 a.m., when the assistant cleaned the ...
Name: Description, Length: 425, dtype: object

<span style="color:green;font-weight:bold; font-size:2em">Preparing a function to clean the tweets or input string. <span>

<span style="color:red;font-weight:bold; font-size:em">Note:- The output of this function will be the a list of words as the sentence will be broken down after cleaning. So we will have to join those words to form the sentence<span>

In [53]:
import string
import re
import os
import nltk

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 

tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

stopwords_english = stopwords.words('english')


def process_tweet(tweet):
    '''
    Input: 
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    
    '''
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    #remove numbers
    tweet = re.sub(r'\d+', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    ### START CODE HERE ###
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and # remove stopwords
            word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            tweets_clean.append(word)
    ### END CODE HERE ###
    return tweets_clean


In [54]:
desc_list = []
for x in df_new['Description']:
        desc_list.append(process_tweet(x))

<span style="color:red;font-weight:bold; font-size:1em">Joining the words to form a sentence<span>

In [55]:
desc_list1 = []
for x in desc_list:
    desc_list1.append(' '.join(x))

In [56]:
desc_list1[0]

'removing drill rod jumbo maintenance supervisor proceeds loosen support intermediate centralizer facilitate removal seeing mechanic supports one end drill equipment pull hands bar accelerate removal moment bar slides point support tightens fingers mechanic drilling bar beam jumbo'

<span style="color:red;font-weight:bold; font-size:1em">Converting the above list into DataFrame<span>

In [57]:
df1 = pd.DataFrame({'New_Description':desc_list1})

In [58]:
df1

,New_Description
0,removing drill rod jumbo maintenance superviso...
1,activation sodium sulphide pump piping uncoupl...
2,sub-station milpo located level collaborator e...
3,approximately nv cx ob personnel begins task u...
4,approximately circumstances mechanics anthony ...
...,...
420,approximately approximately lifting kelly hq t...
421,collaborator moved infrastructure office julio...
422,environmental monitoring activity area employe...
423,employee performed activity stripping cathodes...


<span style="color:green;font-weight:bold; font-size:em">Important thing to note here is that the above dataframe is just the description part of our main DataFrame. So, we need to concat the above DataFrame into our main dataframe, then add all the columns to form one single description <span>

In [59]:
df1 = pd.concat([df_new,df1],axis=1)

In [60]:
df1.head(2)

,Countries,Local,Industry Sector,Genre,Employee or Third Party,Critical Risk,Description,Year,Date,Month_Modified,New_Description
0,Country_01,Local_01,Mining,Male,Third Party,Pressed,While removing the drill rod of the Jumbo 08 f...,2016,1,January,removing drill rod jumbo maintenance superviso...
1,Country_02,Local_02,Mining,Male,Employee,Pressurized Systems,During the activation of a sodium sulphide pum...,2016,2,January,activation sodium sulphide pump piping uncoupl...


In [61]:
df1.drop('Description',axis=1,inplace=True)

In [62]:
df1.head(2)

,Countries,Local,Industry Sector,Genre,Employee or Third Party,Critical Risk,Year,Date,Month_Modified,New_Description
0,Country_01,Local_01,Mining,Male,Third Party,Pressed,2016,1,January,removing drill rod jumbo maintenance superviso...
1,Country_02,Local_02,Mining,Male,Employee,Pressurized Systems,2016,2,January,activation sodium sulphide pump piping uncoupl...


<span style="color:green;font-weight:bold; font-size:1em">Here, we are adding all the values of a dataframe and storing it in the column 'Description'<span>

In [63]:
df1['Description'] = df1.sum(axis=1)
df1.head(2)

,Countries,Local,Industry Sector,Genre,Employee or Third Party,Critical Risk,Year,Date,Month_Modified,New_Description,Description
0,Country_01,Local_01,Mining,Male,Third Party,Pressed,2016,1,January,removing drill rod jumbo maintenance superviso...,Country_01 Local_01 Mining Male Third Party Pr...
1,Country_02,Local_02,Mining,Male,Employee,Pressurized Systems,2016,2,January,activation sodium sulphide pump piping uncoupl...,Country_02 Local_02 Mining Male Employee Press...


<span style="color:green;font-weight:bold; font-size:em">Going forward, this forms our X value. We will convert these words to tensor and feed to the LSTM Model<span>

In [64]:
df1['Description'][0]

'Country_01 Local_01 Mining Male Third Party Pressed 2016 1 January removing drill rod jumbo maintenance supervisor proceeds loosen support intermediate centralizer facilitate removal seeing mechanic supports one end drill equipment pull hands bar accelerate removal moment bar slides point support tightens fingers mechanic drilling bar beam jumbo'

<span style="color:green;font-weight:bold; font-size:2em">Extracting total Vocabulary words<span>

In [65]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val

# Include special tokens 
# started with pad, end of line and unk tokens
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

# Note that we build vocab using training data
for sentence in df1['Description']: 
    for word in sentence.split():
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are",len(Vocab))
display(Vocab)

Total words in vocab are 3173


{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'Country_01': 3,
 'Local_01': 4,
 'Mining': 5,
 'Male': 6,
 'Third': 7,
 'Party': 8,
 'Pressed': 9,
 '2016': 10,
 '1': 11,
 'January': 12,
 'removing': 13,
 'drill': 14,
 'rod': 15,
 'jumbo': 16,
 'maintenance': 17,
 'supervisor': 18,
 'proceeds': 19,
 'loosen': 20,
 'support': 21,
 'intermediate': 22,
 'centralizer': 23,
 'facilitate': 24,
 'removal': 25,
 'seeing': 26,
 'mechanic': 27,
 'supports': 28,
 'one': 29,
 'end': 30,
 'equipment': 31,
 'pull': 32,
 'hands': 33,
 'bar': 34,
 'accelerate': 35,
 'moment': 36,
 'slides': 37,
 'point': 38,
 'tightens': 39,
 'fingers': 40,
 'drilling': 41,
 'beam': 42,
 'Country_02': 43,
 'Local_02': 44,
 'Employee': 45,
 'Pressurized': 46,
 'Systems': 47,
 '2': 48,
 'activation': 49,
 'sodium': 50,
 'sulphide': 51,
 'pump': 52,
 'piping': 53,
 'uncoupled': 54,
 'sulfide': 55,
 'solution': 56,
 'designed': 57,
 'area': 58,
 'reach': 59,
 'maid': 60,
 'immediately': 61,
 'made': 62,
 'use': 63,
 'emerg

In [66]:
Vocab['Local_03']

74

<span style="color:green;font-weight:bold; font-size:2em">Writing a function to convert the description to tensor<span>

In [67]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = tweet.split()
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = Vocab['__UNK__']
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = Vocab[word] if word in Vocab else unk_ID
    ### END CODE HERE ###
        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l


<span style="color:green;font-weight:bold; font-size:2em">Tensors Created<span>

In [68]:
tensor_values=[]
 
for row in df1['Description']:    
           tensor_values.append(tweet_to_tensor(row, Vocab, unk_token='__UNK__', verbose=False))

In [69]:
df1['Description'][99]

'Country_02 Local_05 Metals Male Third Party Pressurized Systems / Chemical Substances 2016 22 April employee reports placed air lance tank opened manual air valve projection acid solution heated toward reaching front left thigh'

In [70]:
y[99]

4

In [71]:
np.array(tensor_values[5])

array([ 43, 182, 183,   6,   7,   8,  75,  46,  47,  10, 184,  12, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 192, 203, 187, 204,  83, 205, 206, 207])

<span style="color:green;font-weight:bold; font-size:2em">Maximum length of the Sentence<span>

In [72]:
max_len = 0

for x in range(len(tensor_values)):
    a = len(tensor_values[x])
    if a > max_len:
        max_len = a

max_len

105

<span style="color:green;font-weight:bold; font-size:2em">Padding the X values<span>

In [73]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 100
X = pad_sequences(maxlen=maxlen, sequences=tensor_values, padding="post", value=0)

<span style="color:green;font-weight:bold; font-size:2em">Building Bidirectional LSTM Model<span>

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(len(Vocab), embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          406144    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 128)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 392)               509600    
_________________________________________________________________
dense (Dense)                (None, 6)                 2358      
Total params: 918,102
Trainable params: 918,102
Non-trainable params: 0
_________________________________________________________________
None


<span style="color:green;font-weight:bold; font-size:2em">Creating Y categorical Data and splitting the data<span>

In [75]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(y).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.25, random_state = 88)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(318, 100) (318, 6)
(107, 100) (107, 6)


In [80]:
batch_size = 20
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 1)

Train on 318 samples
Epoch 1/20
318/318 [==============================] - 6s 18ms/sample - loss: 0.1321 - accuracy: 0.9623
Epoch 2/20
318/318 [==============================] - 3s 11ms/sample - loss: 0.1090 - accuracy: 0.9654
Epoch 3/20
318/318 [==============================] - 3s 11ms/sample - loss: 0.0612 - accuracy: 0.9937
Epoch 4/20
318/318 [==============================] - 4s 11ms/sample - loss: 0.0477 - accuracy: 0.9874
Epoch 5/20
318/318 [==============================] - 4s 14ms/sample - loss: 0.1075 - accuracy: 0.9811
Epoch 6/20
318/318 [==============================] - 4s 11ms/sample - loss: 0.3841 - accuracy: 0.8742
Epoch 7/20
318/318 [==============================] - 4s 11ms/sample - loss: 0.2729 - accuracy: 0.8994
Epoch 8/20
318/318 [==============================] - 4s 12ms/sample - loss: 0.1936 - accuracy: 0.9434
Epoch 9/20
318/318 [==============================] - 4s 12ms/sample - loss: 0.1456 - accuracy: 0.9528
Epoch 10/20
318/318 [==============================]

<span style="color:green;font-weight:bold; font-size:2em">Pickling or Saving the Model<span>

In [81]:
model.save('model.h5')

<span style="color:green;font-weight:bold; font-size:2em">Checking the Test Accuracy<span>

In [82]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

107/1 - 0s - loss: 2.7470 - accuracy: 0.4766
score: 3.17
acc: 0.48


<span style="color:green;font-weight:bold; font-size:2em">Classification Report<span>

In [83]:
from sklearn.metrics import classification_report

pred1 = model.predict(X_test)

print(classification_report(Y_test.argmax(axis=1), pred1.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.91      0.67      0.77        15
           1       0.40      0.46      0.43        26
           2       0.38      0.37      0.38        27
           3       0.50      0.62      0.55        29
           4       0.33      0.10      0.15        10
           5       0.00      0.00      0.00         0

    accuracy                           0.48       107
   macro avg       0.42      0.37      0.38       107
weighted avg       0.49      0.48      0.47       107



C:\Users\Shivang Shrivastav\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
